In [11]:
# rap 모델
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop, Adam
from keras.utils.data_utils import get_file
import numpy as np
import pandas as pd
import random
import sys
import io
import re

In [12]:
lyric = pd.read_csv('music_lyrics_total.csv', encoding = 'utf-8')
#child = pd.read_csv('music_lyrics_children_total.csv', encoding = 'utf-8')

In [13]:
# child = child['lyrics'].drop_duplicates()
# child.shape

In [14]:
# text = ''

# for i in range(2000):
#     if child[i] != None:
#         song = child[i]
# #         print(song)
#         song = re.sub('[\(\)-]', ' ', song)
#         cleaned = song.lower().replace(' ', '\n')
#     #     print(cleaned)
#         text = text + " ".join(re.findall(r"[가-힣]+", cleaned)) + ' '
# len(text)

In [15]:
text2 = ''
for i in range(10000):
    if lyric['lyrics'][i] != None:
        song = lyric['lyrics'][i]
#         print(song)
        song = re.sub('[\(\)-]', ' ', song)
        cleaned = song.lower().replace(' ', '\n')
    #     print(cleaned)
        text2 = text2 + " ".join(re.findall(r"[가-힣]+", cleaned)) + ' '
len(text2)

4245223

In [16]:
text =text2
len(text)

4245223

In [17]:
tokens = re.findall(r"[a-zA-Z가-힣'\s]", text)

chars = sorted(list(set(tokens)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 1800


In [18]:
maxlen = 10
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
    
print('nb sequences:', len(sentences))

nb sequences: 1415071


In [19]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

MemoryError: 

In [ ]:
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
#model.add(Dropout(0.2)) # 추가
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr = 0.01))
model.summary()

In [ ]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end(epoch, logs):
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.3, 0.5]: # 0.2, 0.5
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [ ]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

history = model.fit(
    x, 
    y,
    batch_size=1024,
    epochs=30,
    callbacks=[print_callback]
)

In [ ]:
from keras.models import load_model

model.save('./sample_data/model_child2000_lyric6000_v1.h5') 